In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 7)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

b: tensor([[ 12.4995,  -9.9574,  11.0620, -21.6085,   1.2435, -10.8056,   8.1218]],
       grad_fn=<AddmmBackward>)
epoch [4/200], loss:-1.2562
x:  tensor([[-17.8059, -37.5465, -13.4594, -10.1304, -33.2389, -21.9049,   9.5923]],
       grad_fn=<AddmmBackward>)
y: tensor([[-17.5723, -37.4081, -13.5952, -10.3980, -33.2993, -21.8367,   9.1383]],
       grad_fn=<AddmmBackward>)
b: tensor([[-18.7741, -32.1159, -10.8212,  -9.8735, -30.6169, -21.1715,   8.6450]],
       grad_fn=<AddmmBackward>)
epoch [5/200], loss:-1.2463
x:  tensor([[-44.7870, -25.6582, -25.3855, -12.6368, -58.9097, -24.1934,  18.6194]],
       grad_fn=<AddmmBackward>)
y: tensor([[-44.4752, -25.4344, -24.8845, -11.9716, -57.6810, -23.5507,  19.0815]],
       grad_fn=<AddmmBackward>)
b: tensor([[-43.1850, -20.7336, -22.0358, -11.6873, -54.5754, -23.1581,  17.7398]],
       grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.4551
x:  tensor([[-26.2531,  -4.4077,  -1.4871, -21.8406, -33.4188, -33.0640, -32.9932]],
       grad_fn=<A

epoch [26/200], loss:-1.7686
x:  tensor([[  4.8353,  50.2918,   6.2937,   1.4559,   9.2516, -32.6196,  71.6196]],
       grad_fn=<AddmmBackward>)
y: tensor([[  4.7611,  50.0249,   6.4010,   1.6808,   8.9976, -32.6374,  72.3801]],
       grad_fn=<AddmmBackward>)
b: tensor([[  1.0312,  49.5413,   7.6458,   1.2459,   7.8317, -30.8652,  62.7025]],
       grad_fn=<AddmmBackward>)
epoch [27/200], loss:-1.3898
x:  tensor([[-30.3280,  13.7592,   7.8938, -22.5914, -15.5246, -15.6120,  52.6631]],
       grad_fn=<AddmmBackward>)
y: tensor([[-29.4761,  13.6529,   7.0368, -22.9568, -15.7000, -15.0351,  52.9588]],
       grad_fn=<AddmmBackward>)
b: tensor([[-29.3846,  15.9014,   9.3582, -20.3689, -12.7001, -14.9130,  44.0906]],
       grad_fn=<AddmmBackward>)
epoch [28/200], loss:-1.5340
x:  tensor([[-10.5703,  23.7107,  10.5845, -22.5899, -13.5259, -18.5585,  65.3241]],
       grad_fn=<AddmmBackward>)
y: tensor([[-10.6569,  23.3949,  10.7135, -22.4150, -13.7735, -18.5882,  66.0032]],
       grad_fn

epoch [48/200], loss:-1.5232
x:  tensor([[-47.4836, -17.6372,  -0.5090,  -0.7745,  32.8831,   9.5287, -29.3967]],
       grad_fn=<AddmmBackward>)
y: tensor([[-47.6836, -17.8394,  -0.2553,  -0.8317,  32.7651,   9.5245, -28.6365]],
       grad_fn=<AddmmBackward>)
b: tensor([[-4.4259e+01, -1.1863e+01, -4.3672e-02, -7.8696e-01,  2.9075e+01,
          8.1871e+00, -2.9239e+01]], grad_fn=<AddmmBackward>)
epoch [49/200], loss:-1.4587
x:  tensor([[-22.4739, -20.0075,  -8.2222,  18.2722,   3.5548,   3.6884,   0.5640]],
       grad_fn=<AddmmBackward>)
y: tensor([[-22.3454, -19.4956,  -9.1798,  18.2291,   2.6465,   3.7050,   0.5470]],
       grad_fn=<AddmmBackward>)
b: tensor([[-21.7770, -14.3309,  -6.7503,  15.9739,   3.8211,   3.0005,  -2.2562]],
       grad_fn=<AddmmBackward>)
epoch [50/200], loss:-1.4437
x:  tensor([[-41.9990,  -3.7131,   1.4813,   2.8317,  22.3166,  -0.4781,   1.8765]],
       grad_fn=<AddmmBackward>)
y: tensor([[-42.3241,  -3.5569,   1.4342,   2.8800,  21.8345,  -0.8173,   1

b: tensor([[-10.0038,  12.2153,  -2.6181,  -1.7534,  21.2528, -20.7142,  11.6819]],
       grad_fn=<AddmmBackward>)
epoch [70/200], loss:-1.5617
x:  tensor([[-15.1665,  -4.0269,  -4.2719,   4.3536, -13.6659,  -7.4617,  22.6123]],
       grad_fn=<AddmmBackward>)
y: tensor([[-15.7204,  -3.2863,  -4.0411,   4.4561, -13.3836,  -7.8444,  22.3081]],
       grad_fn=<AddmmBackward>)
b: tensor([[-15.7014,   0.4956,  -3.5772,   3.4451, -11.5567,  -7.5325,  17.0234]],
       grad_fn=<AddmmBackward>)
epoch [71/200], loss:-1.5273
x:  tensor([[ -8.0724,  19.0020,   0.2100,   7.8587,   5.8557, -12.8313,  27.3280]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -7.2635,  18.9285,  -0.6077,   7.8210,   5.6617, -12.3303,  27.5031]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -8.6816,  18.9698,   1.0478,   6.8474,   6.5470, -11.2786,  22.1608]],
       grad_fn=<AddmmBackward>)
epoch [72/200], loss:-1.4536
x:  tensor([[ -3.8128,   4.0094,   9.9977,   9.7384,  17.0089, -11.4566,  30.8376]],
       grad_fn

epoch [92/200], loss:-1.4629
x:  tensor([[ -2.4089,  15.1853,  24.2938,   5.0832, -12.5817,  -2.1481,  -6.1886]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -2.3834,  15.4607,  24.5387,   5.0477, -12.5699,  -1.8583,  -4.9108]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -4.5523,  16.5064,  21.7566,   4.5526, -11.8628,  -2.0272,  -7.0315]],
       grad_fn=<AddmmBackward>)
epoch [93/200], loss:-1.3991
x:  tensor([[-21.6824,  23.0111,  17.6824,   5.6508, -23.0893, -29.6682,  11.8022]],
       grad_fn=<AddmmBackward>)
y: tensor([[-21.3868,  23.5654,  16.6308,   5.6276, -24.0942, -29.5740,  11.6633]],
       grad_fn=<AddmmBackward>)
b: tensor([[-20.2840,  22.6644,  15.6037,   5.0948, -20.7041, -25.8365,   7.8282]],
       grad_fn=<AddmmBackward>)
epoch [94/200], loss:-1.5308
x:  tensor([[-30.1984,   7.4491,  -0.1935,  10.3937, -21.7713, -20.2877, -10.8466]],
       grad_fn=<AddmmBackward>)
y: tensor([[-29.3030,   7.2446,  -1.0409,  10.3208, -21.9546, -19.7150, -10.6201]],
       grad_fn

epoch [114/200], loss:-1.4245
x:  tensor([[-0.7684, 10.0072,  0.3522,  3.5491,  4.7041,  5.5436,  4.0688]],
       grad_fn=<AddmmBackward>)
y: tensor([[-1.2010, 10.0401,  0.2965,  3.5840,  4.1978,  5.1020,  3.3426]],
       grad_fn=<AddmmBackward>)
b: tensor([[-2.8336, 10.4821,  0.5761,  3.5422,  3.5321,  4.2714,  0.7248]],
       grad_fn=<AddmmBackward>)
epoch [115/200], loss:-1.3037
x:  tensor([[-6.2314, 26.1304, -1.8446,  2.3749,  9.3715, -0.9391, 13.0609]],
       grad_fn=<AddmmBackward>)
y: tensor([[-6.1808, 26.0265, -1.4307,  2.3498, 10.8172, -0.9261, 12.7387]],
       grad_fn=<AddmmBackward>)
b: tensor([[-8.5604, 24.8333,  0.1220,  2.7865,  7.8860, -1.9962,  9.0560]],
       grad_fn=<AddmmBackward>)
epoch [116/200], loss:-1.4018
x:  tensor([[-19.1607,  14.9964,  -4.4201,   0.0941,   5.3239,   9.2894,   3.2660]],
       grad_fn=<AddmmBackward>)
y: tensor([[-19.6833,  15.6443,  -4.1904,   0.1214,   5.6156,   8.8786,   2.8686]],
       grad_fn=<AddmmBackward>)
b: tensor([[-19.2965,

epoch [136/200], loss:-1.4169
x:  tensor([[ -4.8983,  15.6851,  11.6783,   4.0507, -16.1122,  -2.2074,   4.8172]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -4.7732,  16.0516,  11.9618,   4.0447, -16.0511,  -1.9564,   5.9388]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -6.1741,  16.9654,  10.4173,   4.2908, -15.5085,  -2.5946,   2.8209]],
       grad_fn=<AddmmBackward>)
epoch [137/200], loss:-1.5774
x:  tensor([[-10.8624,  14.4092,  -1.9623,   0.9235,  -1.9458,  -9.8023,   6.1255]],
       grad_fn=<AddmmBackward>)
y: tensor([[-10.9291,  12.4908,  -1.4178,   0.9299,  -2.4259,  -9.8507,   5.8527]],
       grad_fn=<AddmmBackward>)
b: tensor([[-11.3421,  14.4228,  -0.4734,   1.4759,  -3.4514,  -8.9670,   4.4205]],
       grad_fn=<AddmmBackward>)
epoch [138/200], loss:-1.5634
x:  tensor([[-22.0472,  10.0907,  -0.4826,   1.1445,  -2.6108,  -8.0785,   2.2358]],
       grad_fn=<AddmmBackward>)
y: tensor([[-22.4409,  10.0329,  -0.4720,   1.1632,  -3.0384,  -8.4365,   1.6004]],
       grad

epoch [158/200], loss:-1.6240
x:  tensor([[  2.5021,  -4.3694,  11.7337,   1.3183, -18.5882,  -8.4097,  16.4294]],
       grad_fn=<AddmmBackward>)
y: tensor([[  2.0015,  -3.7426,  11.8873,   1.3220, -18.3083,  -8.7932,  15.9852]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -0.2695,  -2.1615,  10.8346,   2.0858, -18.7696,  -8.0176,  12.7851]],
       grad_fn=<AddmmBackward>)
epoch [159/200], loss:-1.4085
x:  tensor([[-16.4726,   0.5168,   2.4211,   1.5248, -28.0622, -13.4774,  -0.4867]],
       grad_fn=<AddmmBackward>)
y: tensor([[-16.4422,   0.7967,   2.6666,   1.5238, -28.1939, -13.3162,   0.2754]],
       grad_fn=<AddmmBackward>)
b: tensor([[-16.0561,   2.3752,   1.9927,   2.3035, -26.5031, -11.8945,  -2.0981]],
       grad_fn=<AddmmBackward>)
epoch [160/200], loss:-1.3993
x:  tensor([[-6.1883, 13.2877,  4.9897,  1.2519, -9.1744, -8.1980,  2.7207]],
       grad_fn=<AddmmBackward>)
y: tensor([[-6.2594, 13.1388,  5.3374,  1.2796, -7.8471, -8.2807,  2.4064]],
       grad_fn=<AddmmBack

epoch [180/200], loss:-1.5803
x:  tensor([[-15.7517,  19.3158,  12.7686,   1.0432,   0.3583,  -1.4054,  13.9565]],
       grad_fn=<AddmmBackward>)
y: tensor([[-15.4818,  19.6851,  11.8919,   1.0578,  -0.5671,  -1.3529,  13.7075]],
       grad_fn=<AddmmBackward>)
b: tensor([[-14.7221,  19.0231,  10.4576,   2.0111,  -1.1562,  -0.8493,  10.4454]],
       grad_fn=<AddmmBackward>)
epoch [181/200], loss:-1.7250
x:  tensor([[-3.7541,  9.7385,  2.4713,  1.3720, -6.2787, -9.2303, 11.3770]],
       grad_fn=<AddmmBackward>)
y: tensor([[-2.8613,  9.4655,  1.6129,  1.3617, -6.5435, -8.6762, 11.5863]],
       grad_fn=<AddmmBackward>)
b: tensor([[-4.3560,  9.9203,  2.1734,  2.2703, -5.9275, -7.8555,  8.6386]],
       grad_fn=<AddmmBackward>)
epoch [182/200], loss:-1.4418
x:  tensor([[-5.0267,  4.9382, 10.0189,  2.1693, -9.8905, -5.7238, 20.0377]],
       grad_fn=<AddmmBackward>)
y: tensor([[-4.9911,  5.2496, 10.2587,  2.1349, -9.9938, -5.5941, 20.7611]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [2]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time
print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(2):
    z = cv2.imread('Images/0c.png')
    z  = cv2.resize(z,(100,100))
    img = z
    print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


(100, 100, 3)
Length of Class data:  600
Start
(100, 100, 3)
"Drone Stationary, Tank Firing"
(100, 100, 3)
"Drone Stationary, Tank Firing"


In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4